In [10]:
import pandas as pd
from util import OpenAIGPT
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
igpt = OpenAIGPT(keys_path="gpt3keys.txt")

In [11]:
test_data = '../data/flight_review/test.json'
with open(test_data, 'r') as file:
    data = json.load(file)

print(data[0]['input'])

@SouthwestAir you're my early frontrunner for best airline! #oscars2016


### generating gpt's answer


In [12]:
def match_choice(text):
    match = re.findall(r'.*?([A-D]+(?:[、, ]+[A-D]+)*)', text)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ]+', last_match))
    return ''


def process_row(row):
    try:
        instruction = row['instruction']
        input_data = row['input']
        label = row['output']

        res = igpt(f'{instruction}, {input_data}')

        return {
            "label": label,
            "predict": res
        }
    except Exception as e:
        print(f"Error processing row: {e}")
        return None


def generate_dataset(data, max_samples=100):
    global igpt
    igpt = OpenAIGPT(model_name='gpt-3.5-turbo', keys_path='gpt3keys.txt')
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_row, row) for row in data[:max_samples]]
        for future in tqdm(futures, total=len(futures), desc="Processing rows"):
            result = future.result()
            if result:
                results.append(result)

    return results

In [13]:
dataset = generate_dataset(data, max_samples=100)

Processing rows: 100%|██████████| 100/100 [02:13<00:00,  1.34s/it]


In [14]:
output_file_path = '../saves/exp/GPT-3.5/sentiment_test/generated_predictions.jsonl'
with open(output_file_path, 'w', encoding='utf-8') as f:
    for item in dataset:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

### score gpt's answer


In [15]:
import json
from sklearn.metrics import accuracy_score, f1_score
import re


def match_choice(text):
    match = re.findall(r'.*?([A-D]+(?:[、, ]+[A-D]+)*)', text)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ]+', last_match))
    return ''


def accuracy(path):
    y_true = []
    y_pred = []
    with open(path, 'r') as file:
        for line in file:
            data = json.loads(line)
            y_true.append(match_choice(data['label']))
            y_pred.append(match_choice(data['predict']))
    return accuracy_score(y_true, y_pred)


def score(path):
    y_true = []
    y_pred = []
    with open(path, 'r') as file:
        for line in file:
            data = json.loads(line)
            y_true.append(match_choice(data['label']))
            y_pred.append(match_choice(data['predict']))
    return f1_score(y_true, y_pred, average='macro')


def out(path):
    print(f'acc: {accuracy(path)}')
    print(f'f1: {score(path)}')

In [16]:
path = '../saves/exp/GPT-3.5/travel_insurance_4_label_test/generated_predictions.jsonl'
out(path)

acc: 0.4
f1: 0.18355933721450965
f1: 0.18355933721450965
